In [1]:
import pandas as pd
import pickle 
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
import requests
import urllib
import os
import zipfile


In [60]:
os.chdir('/work/notebook/')

# 1. Data Download


In [61]:
def download_applications(year):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:57.0) Gecko"\
               "/20100101 Firefox/57.0",}

    lnk = 'https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/'\
          + str(year) + '/'
    rr = requests.get(lnk, headers=headers)
    html = rr.content
    soup = BeautifulSoup(html, "html.parser")
    urls = [x.get('href') for x in soup.find_all('a') 
            if '.zip' in x.get('href')]
    target_dir = '../data/apps/' + str(year) + '/'
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    [urllib.request.urlretrieve(lnk + x, 
                                '../data/apps/' + str(year) + '/'+x) for
     x in urls[0:2]]

In [62]:
year = 2017
download_applications(year)
olddir = os.getcwd()
newdir = '/work/data/apps/' + str(year) + '/'
os.chdir(newdir)



In [96]:
def unzip_file(fnm):
    zip_ref = zipfile.ZipFile(fnm, 'r')
    zip_ref.extractall('.')
    zip_ref.close()
    return fnm.split('/')[-1].replace('.zip', '.xml')

In [116]:
%%time
for fnm in os.listdir():
    fnm_xml = unzip_file(fnm)
    print(fnm_xml)
    !bash /work/split_apps.sh {fnm_xml}
    !rm {fnm}

ipa170330.zip
ipa170330.xml
ipa170119.zip
ipa170119.xml
ipa170316.zip
ipa170316.xml
ipa170223.zip
ipa170223.xml
ipa170413.zip
ipa170413.xml
ipa170518.zip
ipa170518.xml
ipa170427.zip
ipa170427.xml
ipa170713.zip
ipa170713.xml
ipa170302.zip
ipa170302.xml
ipa170601.zip
ipa170601.xml
ipa170406.zip
ipa170406.xml
ipa170706.zip
ipa170706.xml
ipa170629.zip
ipa170629.xml
ipa170504.zip
ipa170504.xml
ipa170105.zip
ipa170105.xml
ipa170622.zip
ipa170622.xml
ipa170202.zip
ipa170202.xml
ipa170126.zip
ipa170126.xml
ipa170608.zip
ipa170608.xml
ipa170420.zip
ipa170420.xml
CPU times: user 1min 13s, sys: 13 s, total: 1min 26s
Wall time: 3min 54s


In [115]:
% rm ipa170727.zip

# 2. Find doc-numbers

In [86]:
def get_docnumbers(fnm):
    try:
        tree = ET.parse(fnm)
        dc_nums = ["".join([x for x in numb.itertext()]) for numb in 
                                           tree.iter(tag = 'doc-number')]
        return pd.DataFrame.from_dict({fnm.split('/')[-1] : dc_nums}).T
    except:
        return pd.DataFrame.from_dict({fnm.split('/')[-1] : [-1111]}).T
    
    
def find_app_ids(file_dir):
    rows = [get_docnumbers(file_dir + x) for x in 
                                     os.listdir(file_dir)]
    pdrows = pd.concat(rows)
    pdrows.index = file_dir + pdrows.index
    return pdrows

In [ ]:
all_app_ids = pd.concat([find_app_ids(newdir + x +'/') 
                              for x in os.listdir(newdir)])
all_app_ids = all_app_ids.rename(columns={1: 'app_id'})

In [117]:
all_app_ids.loc[all_app_ids.app_id.isnull()]

,0,app_id,2,3,4,5,6,7,8,9,...,353,354,355,356,357,358,359,360,361,362
/work/data/apps/2016/ipa160114/F_941.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_60.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_2567.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_902.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_3654.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_927.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_2276.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_915.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_913.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/work/data/apps/2016/ipa160114/F_903.xml,-1111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
!cat /work/data/apps/2016/ipa160114/F_941.xml









































































































































































































































































































































































































































































































































































</us-patent-application>


In [ ]:
fnm = '/work/data/apps/2016/ipa160114/F_941.xml'
tree = ET.parse(fnm)